# Inverted Pendulum using DQN

When moving on from discrete environments, such as grid world or black jack, the amount of state in a continous environment renders traditional tabular methods (Monte Carlo and TD learning) intractable (at least without any modification to the problem formulation). DQN method addresses this problem by using a value function approximator, essentially replacing the Q-table of explicit state-action value with a parameterized function $q^\pi_{\vec w}(s,a,\vec w)$. The vector $\vec w$ stores the weights or parameters of this approximating function.

The motivation of using value function approximator is clear, but where does it fit into the RL picture?
- Value function approximator is a direct mapping between the states (features) to the state-action value $Q(s,\cdot)$ 
- As such, the approximator function fits in the policy evaluation step of the RL training process. We train this function by minimizing the loss between the approximated value function and the actual value function $ s.t. \enspace q^\pi_{\vec w}(s,a,\vec w) \approx q^*$
- However, the true action value function $q^*$ is unknown and must be learnt from the interaction between the agent and its environment. We must employ Q-learning to approximate the true action value function, toward which we will update the parameterized value function $q^\pi_{\vec w}(s,a,\vec w)$. The update is as follow

**The DQN algorithm**
1. From a state $s$, take action $a$ using $\epsilon$-greedy policy, observe the reward $r$ and next state $s'$
2. Store the transition tuple $(s, a, r, s')$ in a replay memory $\mathcal{D}$. This replay memory is a buffer that stores the most recent transition tuples
3. From this replay memory, sample batches of $(s_t, a_t, r_{t+1}, s_{t+1})$ randomly for training the Q-function approximator
4. Adjust the parameters $\vec w$ by optimizing the mean squared error (MSE) between the Q network and the Q-learning target using Q-learning
$$ \mathcal{L}_i (\vec w_i) = \mathbb{E}_{(s,a,r,s') \sim \mathcal{D}} \bigg[ \Big(r + \gamma \max_{a'}Q(s',a',\vec w^-_i) - Q(s,a,\vec w_i) \Big)^2\bigg]$$ 

When applying stochastic gradient descent, the update at every step is reduced as follows (the math found in the GoodNote notebook):

$$\Delta \vec w = \alpha \Big(r + \gamma \max_{a'} \underbrace{Q(s', a', \vec{w}_i^-)}_{\text{target}} - \underbrace{Q(s,a,\vec w_i)}_{\text{behaviour}} \Big) \vec x(s,a)$$


**Notes:** 

There are two mechanisms introduced in this paper that allow for stable training, one is experience replay, and the other is training two separate methods.
- **Experience replay** - this decorrelates the experience tuples (s,a,r,s') thus follow the i.i.d assumption for training a neural network
- **Two Q-networks**, a target network and a behaviour network. The parameters $\vec w'$ of the target network $Q(s,a,\vec w')$ are only updated infrequently to maintain a stable target. The parameters $\vec w$ of the behaviour network $Q(s,a,\vec w)$ are updated every policy improvement iteration to move toward this target network. The frequency of updating the target network is annother hyperparameter to tune. 


## Question to self
1. What does it mean to apply stochastic gradient descent to simplify the loss function? What are the implications?
- Every policy evaluation iteration, a random sample, or batch of samples, is used for updating the state-action value instead of every single experience stored in the replay buffer. Recall that the random sampling from the replay buffer de-correlates the experience and enforces the i.i.d. assumption
- 


2. Would it be beneficial to formulate the Q network such that the input is the state vector and the outputs are the Q-values associated with each possible action from the state. As such, one can just take the max value from the target Q network and index into the behaviour network for the corresponding action a?


3. At first do I have to run the agent several times using the initial $\epsilon$-greedy policy to fill the memory buffer with training samples?
- Yes, and then pop in new experience at the end, push the oldest experience in the replay buffer out.
